# Get reviews and related information from Australian Taxation Office locations

In [13]:
import googlemaps
import pandas as pd
import time
import os
from retrying import retry
# get the current working directory
os.getcwd()


# define place id constants
ATO_PLACE_IDs = 'ChIJF_JXPNhd1moRrKDkeMhjAgQ'

'/Users/tnathu-ai/VSCode/Government-Digital-Transformation/notebooks/analysis'

In [6]:
GMAPS_KEY = os.getenv("GMAPS_KEY")  # API Key for OpenAI
gmaps = googlemaps.Client(key=GMAPS_KEY)

# Location name to search for
query = 'Australian Taxation Office'
result = gmaps.places(query=query)

# Assuming the first result is the desired one, extract its place_id
if result['results']:
    place_id = result['results'][0]['place_id']
    print(place_id)
else:
    print("No results found")

ChIJF_JXPNhd1moRrKDkeMhjAgQ


In [7]:

def get_place_details(place_id):
    """Get place details."""
    # The API only allows fetching a subset of reviews, not all of them.
    place_details = gmaps.place(place_id=place_id, fields=['name', 'adr_address', 'website', 'reviews', 'rating'])
    return place_details

place_id = ATO_PLACE_IDs

# Fetch place details
details = get_place_details(place_id)

# Extract reviews
reviews = details['result'].get('reviews', [])

# Convert the reviews to a tabular format
df = pd.DataFrame(reviews)

df


,author_name,author_url,language,original_language,profile_photo_url,rating,relative_time_description,text,time,translated
0,Avec With,https://www.google.com/maps/contrib/1134230974...,en,en,https://lh3.googleusercontent.com/a/ACg8ocL7bX...,1,a month ago,The ATO is like the Mafia. They screw you over...,1709800415,False
1,Zakir Ahmed,https://www.google.com/maps/contrib/1001444960...,en,en,https://lh3.googleusercontent.com/a-/ALV-UjXy9...,1,a month ago,ATO don't deserve any star. I had worst experi...,1707786816,False
2,Shine An,https://www.google.com/maps/contrib/1067999549...,en,en,https://lh3.googleusercontent.com/a/ACg8ocLtYV...,1,5 months ago,ATO don't deserve any star. I had worst experi...,1699005129,False
3,Jay,https://www.google.com/maps/contrib/1032598375...,en,en,https://lh3.googleusercontent.com/a/ACg8ocIv7t...,1,7 months ago,2023 and we are still using these stupid hour ...,1693558901,False
4,Aaron Rowe,https://www.google.com/maps/contrib/1132385301...,en,en,https://lh3.googleusercontent.com/a/ACg8ocLO9B...,1,a week ago,I have been attempting to call them over numer...,1711505078,False


In [8]:

def get_place_details(place_id):
    """Get place details for a given place_id."""
    place_details = gmaps.place(place_id=place_id, fields=['name', 'adr_address', 'website', 'reviews', 'rating'])
    return place_details

# Search for the place
query = 'Australian Taxation Office'
search_result = gmaps.places(query=query)

# Check if we have results and extract the place_id
if search_result['results']:
    place_ids = [place['place_id'] for place in search_result['results']]
else:
    place_ids = []
    print("No results found")

# Initialize an empty list to store place details
place_details_list = []

# Iterate over each place_id to fetch its details
for idx, place_id in enumerate(place_ids):
    # Fetch place details and add to the list
    details = get_place_details(place_id)
    place_details_list.append(details)

    # Delay the next API call by 1 second to avoid hitting the rate limit
    # Adjust the sleep time based on specific rate limit
    time.sleep(1)

# Convert the list of place details to a DataFrame
df = pd.DataFrame([detail['result'] for detail in place_details_list if 'result' in detail])

df


,adr_address,name,rating,reviews,website
0,"ATO self service facilities available only, <s...",Australian Taxation Office,1.2,"[{'author_name': 'Avec With', 'author_url': 'h...",https://www.ato.gov.au/
1,"<span class=""street-address"">6 Gladstone St</s...",Australian Taxation Office,2.5,"[{'author_name': 'Christopher Stenton', 'autho...",http://www.ato.gov.au/
2,"<span class=""street-address"">19 George St</spa...",ATO Dandenong,1.3,"[{'author_name': 'Lily', 'author_url': 'https:...",NaN


In [16]:
# export df to csv
df.to_csv('../../data/raw/google_map/ATO_GG_reviews.csv', index=False)
print(f'Exported to {os.getcwd()}/../../data/raw/google_map/ATO_GG_reviews.csv')

Exported to /Users/tnathu-ai/VSCode/Government-Digital-Transformation/notebooks/analysis/../../data/raw/google_map/ATO_GG_reviews.csv


-----> OBSERVATION

+ For the first place (Australian Taxation Office in Footscray):

+ There are 5 reviews listed for this location.
+ For the second place (ATO Dandenong):

+ There are 4 reviews listed for this location.
+ For the third place (Australian Taxation Office (No Public Access) in Docklands):

+ There are 5 reviews listed for this location.

# Consolidated code

In [4]:
import googlemaps
import pandas as pd
import os
import time

# Initialize Google Maps client
def initialize_gmaps(api_key):
    return googlemaps.Client(key=api_key)

# Get details of a specific place
def get_place_details(gmaps, place_id, fields=['name', 'adr_address', 'website', 'reviews', 'rating']):
    return gmaps.place(place_id=place_id, fields=fields)

# Main function to fetch reviews and export to CSV
def fetch_and_save_reviews(place_name, save_directory):
    GMAPS_KEY = os.getenv("GMAPS_KEY")
    gmaps = initialize_gmaps(GMAPS_KEY)

    search_result = gmaps.places(query=place_name)

    if search_result['results']:
        place_ids = [place['place_id'] for place in search_result['results']]
    else:
        print("No results found")
        return

    place_details_list = []
    for place_id in place_ids:
        details = get_place_details(gmaps, place_id)
        place_details_list.append(details)
        time.sleep(1)  # Prevent hitting API rate limit

    # Filter out invalid entries and create DataFrame
    valid_details = [detail['result'] for detail in place_details_list if 'result' in detail]
    if valid_details:
        df = pd.DataFrame(valid_details)
        save_path = os.path.join(save_directory, f"{place_name.replace(' ', '_')}_reviews.csv")
        df.to_csv(save_path, index=False)
        print(f"Exported to {save_path}")
    else:
        print("No valid place details found.")


Exported to ../../data/raw/google_map/Australian_Taxation_Office_reviews.csv


In [ ]:
%%time 
fetch_and_save_reviews("Australian Taxation Office", "../../data/raw/google_map")

In [5]:
fetch_and_save_reviews("Australia Post", "../../data/raw/google_map")

Exported to ../../data/raw/google_map/Australia_Post_reviews.csv


In [6]:
fetch_and_save_reviews("Public Transport Victoria", "../../data/raw/google_map")

Exported to ../../data/raw/google_map/Public_Transport_Victoria_reviews.csv


In [7]:
%%time
fetch_and_save_reviews("Service NSW", "../../data/raw/google_map")

Exported to ../../data/raw/google_map/Service_NSW_reviews.csv
CPU times: user 170 ms, sys: 18.6 ms, total: 189 ms
Wall time: 25.6 s


In [8]:
%%time
fetch_and_save_reviews("SunSmart", "../../data/raw/google_map")

Exported to ../../data/raw/google_map/SunSmart_reviews.csv
CPU times: user 110 ms, sys: 22.3 ms, total: 133 ms
Wall time: 11.6 s
